## Hyperparameter Tuning

In this exercise you will be building a Neural network for which, you will be tuning the **Model Parameters** to find out the parameters with which the model perform its best.

You will using 

1. `Grid Search`
2. `Random Search`


### 1. Import the Packages

In [1]:
import os
import pandas as pd
import wrangle as wr
from numpy import nan

from keras.utils import to_categorical
from keras.wrappers.scikit_learn import KerasClassifier


from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dropout, Flatten, Dense

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV


/home/user/.local/lib/python3.5/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
Using TensorFlow backend.
/home/user/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/user/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/user/.local/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWa

In [2]:
#Read the dataset with pandas
df = pd.read_csv("data2.csv")

In [3]:
df.head()

id diagnosis  radius_mean  texture_mean  perimeter_mean  area_mean  \
0    842302         M        17.99         10.38          122.80     1001.0   
1    842517         M        20.57         17.77          132.90     1326.0   
2  84300903         M        19.69         21.25          130.00     1203.0   
3  84348301         M        11.42         20.38           77.58      386.1   
4  84358402         M        20.29         14.34          135.10     1297.0   

   smoothness_mean  compactness_mean  concavity_mean  concave points_mean  \
0          0.11840           0.27760          0.3001              0.14710   
1          0.08474           0.07864          0.0869              0.07017   
2          0.10960           0.15990          0.1974              0.12790   
3          0.14250           0.28390          0.2414              0.10520   
4          0.10030           0.13280          0.1980              0.10430   

   ...  texture_worst  perimeter_worst  area_worst  smoothness_worst  \
0  ...          17.33           184.60      2019.0            0.1622   
1  ...          23.41           158.80      1956.0            0.1238   
2  ...          25.53           152.50      1709.0            0.1444   
3  ...          26.50            98.87       567.7            0.2098   
4  ...          16.67           152.20      1575.0            0.1374   

   compactness_worst  concavity_worst  concave points_worst  symmetry_worst  \
0             0.6656           0.7119                0.2654          0.4601   
1             0.1866           0.2416                0.1860          0.2750   
2             0.4245           0.4504                0.2430          0.3613   
3             0.8663           0.6869                0.2575          0.6638   
4             0.2050           0.4000                0.1625          0.2364   

   fractal_dimension_worst  Unnamed: 32  
0                  0.11890          NaN  
1                  0.08902          NaN  
2                  0.08758          NaN  
3                  0.17300          NaN  
4                  0.07678          NaN  

[5 rows x 33 columns]

### 2.Basic Data Cleaning


1.   Drop the Unnamed:32` and `id` columns
2.   Consider `diagnosis` column as the labels(y) while the rest of the columns as features (X) 

**Note:**

Convert the labels in terms of 0 and 1 where 1 corresponds to M and 0 Corresponds to 0



In [4]:
def breast_cancer(df):
    '''Load and preprocess(cleaning) the dataset
    Input: Dataframe
    Output: x,y
    x:Features
    y:Labels in form of 0 and 1
    '''
    df.drop(['Unnamed: 32',"id"], axis=1, inplace=True)
    x = df.drop("diagnosis", axis=1)
    y = df["diagnosis"]
   

    return x, y

In [5]:
#Call the Datacleaning Function
x, y = breast_cancer(df)

# Normalize every feature in x to mean 0, std 1 with wrangle rescale_meanzero function
x = wr.df_rescale_meanzero(pd.DataFrame(x)).values



#Initialise the input feature dimension
input_dim = x.shape[1]

In [6]:
input_dim

30

### 3.Decide on the Parameters to be tuned and create the model
We will be creating a 2-layer Neural Network.

In this example we will be tuning only the model parameters while the hyperparameters can be tuned in later exercise.


**Model Parameters to be tuned**
1. `first_neurons`:Number of neurons in the First layer
2. `activation`: Activation function to be used in First layer.
3. `kernel_initializer`:Initializer in both the layers
4. `optimizer`:Optimizer to be used when compiling the model.


 
 **Hyper Parameters**
 1. `epochs`
 2. `batch_size`
 3. `dropout_rate`

 
----------------------------------------------------------------
**Create the array of values for each  parameters**
1. first_neurons with values 8,9
2. activation with values relu and tanh
3. kernel_initializer with values uniform,he_uniform
4. optimizer with values Adam and SGD



**Note: Make sure to initialize the values in the same order**

In [7]:
# Model Design Components
first_neurons = [8,9]
activation =  ["relu","tanh"]
kernel_initializer = ["uniform","he_uniform"]
optimizer = ["adam","SGD"]


# Hyperparameters
epochs = [10]
batch_size = [1024]
dropout_rate = [0.0]

### 4.Creating Model

In [8]:


# Function to create model, required for KerasClassifier
def create_model(first_neuron=9,activation='relu', kernel_initializer='uniform',
                 dropout_rate=0,optimizer='Adam'):
  

    '''

    Input: Model params and Hyper Params to be tuned
    Output: Compiled model

    '''
    
    #1.Create sequential model

    model = Sequential()

    #2. Add the First Dense layer with params 
    #first_neuron,input_dim,kernel_initailizer,activation assigned values from actual function parameters
    
    model.add(Dense(first_neuron, input_dim=input_dim, activation=activation,kernel_initializer=kernel_initializer))
    

    #3. Add dropout to the with the value from actual function parameter dropout_rate

    model.add(Dropout(rate =dropout_rate)                )


    #4. Add the Second Dense layer with params
    #Number of neurons =1
    #Kernel_initializer from function parameter
    #activation=sigmoid


    model.add(Dense(1,activation="sigmoid",kernel_initializer=kernel_initializer))


    #5.Compile model with
    #loss='binary_crossentropy'
    #optimizer from function parameter
    #metrics=accuracy

    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=["accuracy"]                      )
    

    return model

### 5. Create a Keras Classifier

In [9]:
model = KerasClassifier(build_fn=create_model)


### 6. Hyperparameter Tuning 1 - Grid Search
1. Create a GridSearchCV model with parameters
    - Estimator
    - Param_grid
    - n_jobs=1
    - cv=3
    - verbose=2

2. Fit the model with x,y
  

In [10]:
#parameter grid
param_grid = dict(epochs=epochs, 
                  batch_size=batch_size, 
                  optimizer=optimizer,
                  dropout_rate=dropout_rate,
                  activation=activation,
                  kernel_initializer=kernel_initializer,
                  first_neuron=first_neurons)

In [11]:
#create GridSearchCv model
grid = GridSearchCV(model, param_grid, n_jobs=1, cv=3, verbose=2)

#Fit the model and return the result
grid_result = grid.fit(x,y)

Fitting 3 folds for each of 16 candidates, totalling 48 fits
[CV] kernel_initializer=uniform, epochs=10, activation=relu, first_neuron=8, dropout_rate=0.0, optimizer=adam, batch_size=1024 
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
379/379 [==============================] - 0s 465us/sample - loss: 0.6935 - acc: 0.3905
Epoch 2/10
379/379 [==============================] - 0s 8us/sample - loss: 0.6925 - acc: 0.5884
Epoch 3/10
379/379 [==============================] - 0s 7us/sample - loss: 0.6916 - acc: 0.7388
Epoch 4/10
379/379 [==============================] - 0s 7us/sample - loss: 0.6906 - acc: 0.8311
Epoch 5/10
379/379 [==============================] - 0s 7us/sample - loss: 0.6896 - acc: 0.8813
Epoch 6/10
379/379 [==============================] - 0s 7us/sample - loss: 0.6886 - acc:

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.1s remaining:    0.0s


Epoch 1/10
379/379 [==============================] - 0s 469us/sample - loss: 0.6943 - acc: 0.4248
Epoch 2/10
379/379 [==============================] - 0s 8us/sample - loss: 0.6933 - acc: 0.4855
Epoch 3/10
379/379 [==============================] - 0s 7us/sample - loss: 0.6923 - acc: 0.5805
Epoch 4/10
379/379 [==============================] - 0s 8us/sample - loss: 0.6913 - acc: 0.6702
Epoch 5/10
379/379 [==============================] - 0s 7us/sample - loss: 0.6902 - acc: 0.7361
Epoch 6/10
379/379 [==============================] - 0s 8us/sample - loss: 0.6891 - acc: 0.7916
Epoch 7/10
379/379 [==============================] - 0s 8us/sample - loss: 0.6880 - acc: 0.8232
Epoch 8/10
379/379 [==============================] - 0s 7us/sample - loss: 0.6868 - acc: 0.8443
Epoch 9/10
379/379 [==============================] - 0s 6us/sample - loss: 0.6856 - acc: 0.8654
Epoch 10/10
379/379 [==============================] - 0s 3us/sample - loss: 0.6832 - acc: 0.8918
[CV]  kernel_initializer=un

[Parallel(n_jobs=1)]: Done  48 out of  48 | elapsed:  3.9min finished


Epoch 1/10
569/569 [==============================] - 1s 1ms/sample - loss: 0.6921 - acc: 0.6977
Epoch 2/10
569/569 [==============================] - 0s 6us/sample - loss: 0.6914 - acc: 0.7786
Epoch 3/10
569/569 [==============================] - 0s 6us/sample - loss: 0.6906 - acc: 0.8049
Epoch 4/10
569/569 [==============================] - 0s 6us/sample - loss: 0.6899 - acc: 0.8243
Epoch 5/10
569/569 [==============================] - 0s 6us/sample - loss: 0.6890 - acc: 0.8418
Epoch 6/10
569/569 [==============================] - 0s 6us/sample - loss: 0.6882 - acc: 0.8489
Epoch 7/10
569/569 [==============================] - 0s 6us/sample - loss: 0.6872 - acc: 0.8682
Epoch 8/10
569/569 [==============================] - 0s 5us/sample - loss: 0.6863 - acc: 0.8787
Epoch 9/10
569/569 [==============================] - 0s 6us/sample - loss: 0.6853 - acc: 0.8893
Epoch 10/10
569/569 [==============================] - 0s 5us/sample - loss: 0.6842 - acc: 0.8981


In [12]:
#Print the Best Params
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

#Explore the others
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.898067 using {'kernel_initializer': 'uniform', 'epochs': 10, 'activation': 'relu', 'first_neuron': 8, 'dropout_rate': 0.0, 'optimizer': 'adam', 'batch_size': 1024}
0.898067 (0.021448) with: {'kernel_initializer': 'uniform', 'epochs': 10, 'activation': 'relu', 'first_neuron': 8, 'dropout_rate': 0.0, 'optimizer': 'adam', 'batch_size': 1024}
0.646749 (0.139641) with: {'kernel_initializer': 'uniform', 'epochs': 10, 'activation': 'relu', 'first_neuron': 8, 'dropout_rate': 0.0, 'optimizer': 'SGD', 'batch_size': 1024}
0.567663 (0.243270) with: {'kernel_initializer': 'he_uniform', 'epochs': 10, 'activation': 'relu', 'first_neuron': 8, 'dropout_rate': 0.0, 'optimizer': 'adam', 'batch_size': 1024}
0.681898 (0.036398) with: {'kernel_initializer': 'he_uniform', 'epochs': 10, 'activation': 'relu', 'first_neuron': 8, 'dropout_rate': 0.0, 'optimizer': 'SGD', 'batch_size': 1024}
0.885765 (0.059499) with: {'kernel_initializer': 'uniform', 'epochs': 10, 'activation': 'relu', 'first_neuron': 9, '

### 7. Hyperparameter Tuning 1 - Randomized Search
1. Create a RandomizedSearchCV model with parameters
    - Estimator as model
    - param_dist
    - n_iter=8
    - n_jobs=1
    - cv=3
    - verbose=2

2. Fit the model with x,y
  

In [13]:
param_dist = dict(epochs=epochs, 
                  batch_size=batch_size, 
                  optimizer=optimizer,
                  dropout_rate=dropout_rate,
                  activation=activation,
                  kernel_initializer=kernel_initializer,
                  first_neuron=first_neurons)

In [ ]:
#Create the randomsearccv model
random_search = RandomizedSearchCV(model, param_dist, n_iter=8, n_jobs=1, cv=3, verbose=2)

#Fit the model
random_search = random_search.fit(x,y)

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] kernel_initializer=he_uniform, epochs=10, optimizer=adam, first_neuron=8, dropout_rate=0.0, activation=relu, batch_size=1024 
Epoch 1/10
379/379 [==============================] - 1s 2ms/sample - loss: 0.7703 - acc: 0.5646
Epoch 2/10
379/379 [==============================] - 0s 10us/sample - loss: 0.7553 - acc: 0.5752
Epoch 3/10
379/379 [==============================] - 0s 10us/sample - loss: 0.7406 - acc: 0.5805
Epoch 4/10
379/379 [==============================] - 0s 9us/sample - loss: 0.7262 - acc: 0.5910
Epoch 5/10
379/379 [==============================] - 0s 10us/sample - loss: 0.7121 - acc: 0.5963
Epoch 6/10
379/379 [==============================] - 0s 8us/sample - loss: 0.6984 - acc: 0.6042
Epoch 7/10
379/379 [==============================] - 0s 8us/sample - loss: 0.6850 - acc: 0.6121
Epoch 8/10
379/379 [==============================] - 0s 10us/sample - loss: 0.6719 - acc: 0.6174
Epoch 9/10
379/379 [=========

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   10.2s remaining:    0.0s


Epoch 1/10
379/379 [==============================] - 1s 2ms/sample - loss: 0.7443 - acc: 0.6412
Epoch 2/10
379/379 [==============================] - 0s 11us/sample - loss: 0.7287 - acc: 0.6517
Epoch 3/10
379/379 [==============================] - 0s 12us/sample - loss: 0.7135 - acc: 0.6728
Epoch 4/10
379/379 [==============================] - 0s 12us/sample - loss: 0.6986 - acc: 0.6807
Epoch 5/10
379/379 [==============================] - 0s 11us/sample - loss: 0.6842 - acc: 0.6860
Epoch 6/10
379/379 [==============================] - 0s 12us/sample - loss: 0.6701 - acc: 0.7018
Epoch 7/10
379/379 [==============================] - 0s 12us/sample - loss: 0.6563 - acc: 0.7071
Epoch 8/10
379/379 [==============================] - 0s 10us/sample - loss: 0.6429 - acc: 0.7230
Epoch 9/10
379/379 [==============================] - 0s 10us/sample - loss: 0.6298 - acc: 0.7414
Epoch 10/10
379/379 [==============================] - 0s 5us/sample - loss: 0.6047 - acc: 0.7625
[CV]  kernel_initiali

In [ ]:
# Print the best params 
print("Best: %f using %s" % (random_search.best_score_, random_search.best_params_))


#Explore the others
means = random_search.cv_results_['mean_test_score']
stds = random_search.cv_results_['std_test_score']
params = random_search.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

### Save your answers by running the cell below

In [ ]:
import pickle
with open('grid1.pkl', 'wb') as handle:
  pickle.dump(grid.param_grid, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('grid2.pkl', 'wb') as handle:
  pickle.dump(grid.n_jobs, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('grid3.pkl', 'wb') as handle:
  pickle.dump((grid.classes_).tolist(), handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('ran1.pkl', 'wb') as handle:
  pickle.dump(random_search.param_distributions, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('ran2.pkl', 'wb') as handle:
  pickle.dump(random_search.n_iter, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('ran3.pkl', 'wb') as handle:
  pickle.dump(random_search.n_splits_, handle, protocol=pickle.HIGHEST_PROTOCOL)

save_model=create_model()
save_model.save('model.h5')


Don't stop! your learning ! Tune more to explore more.

1. Tune the activations with other values like 'sigmoid','hard_sigmoid','linear',etc.


2. Tune the Kernel initializers with values like normal and zero


3. Tune the Optimizers with RMSprop, Adamax etc

